# 仮想通貨アービトラージ分析

このノートブックでは、取引所間の価格差とアービトラージ機会を分析します。

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pytz
from decimal import Decimal
from sqlalchemy import func
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートをパスに追加
sys.path.append(str(Path.cwd().parent))

from src.database.connection import db
from src.database.models import Exchange, CurrencyPair, PriceTick, ArbitrageOpportunity

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
sns.set_palette("husl")

# 設定
jst = pytz.timezone('Asia/Tokyo')
PAIR_SYMBOL = 'BTC/JPY'

print("ライブラリのインポート完了")

## 1. 基本データの取得

In [ ]:
# 取引所情報
with db.get_session() as session:
    exchanges = session.query(Exchange).filter_by(is_active=True).all()
    exchange_dict = {ex.id: ex.name for ex in exchanges}
    
    # 通貨ペア情報
    pair = session.query(CurrencyPair).filter_by(symbol=PAIR_SYMBOL).first()
    
    print(f"アクティブな取引所数: {len(exchanges)}")
    print(f"取引所: {', '.join([ex.name for ex in exchanges if ex.code != 'binance'])}")
    print(f"通貨ペア: {pair.symbol if pair else 'Not found'}")

## 2. 現在の価格状況

In [ ]:
def get_current_prices(pair_id, minutes_ago=5):
    """現在の価格を取得"""
    with db.get_session() as session:
        time_threshold = datetime.now(jst) - timedelta(minutes=minutes_ago)
        
        prices = []
        for exchange in exchanges:
            if exchange.code == 'binance':
                continue
                
            latest_tick = session.query(PriceTick).filter_by(
                exchange_id=exchange.id,
                pair_id=pair_id
            ).order_by(PriceTick.timestamp.desc()).first()
            
            if latest_tick and latest_tick.timestamp > time_threshold:
                prices.append({
                    'exchange': exchange.name,
                    'exchange_id': exchange.id,
                    'bid': float(latest_tick.bid),
                    'ask': float(latest_tick.ask),
                    'timestamp': latest_tick.timestamp
                })
        
        return pd.DataFrame(prices)

# 現在の価格を取得
current_prices = get_current_prices(pair.id)
current_prices['spread'] = current_prices['ask'] - current_prices['bid']
current_prices['spread_pct'] = (current_prices['spread'] / current_prices['bid']) * 100

print("現在の価格:")
display(current_prices[['exchange', 'bid', 'ask', 'spread', 'spread_pct', 'timestamp']].round(2))

## 3. 価格差の可視化

In [ ]:
if not current_prices.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Bid価格の比較
    axes[0, 0].bar(current_prices['exchange'], current_prices['bid'], color='lightblue')
    axes[0, 0].set_title('Bid Price by Exchange')
    axes[0, 0].set_ylabel('Price (JPY)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Ask価格の比較
    axes[0, 1].bar(current_prices['exchange'], current_prices['ask'], color='lightcoral')
    axes[0, 1].set_title('Ask Price by Exchange')
    axes[0, 1].set_ylabel('Price (JPY)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. スプレッドの比較
    axes[1, 0].bar(current_prices['exchange'], current_prices['spread_pct'], color='lightgreen')
    axes[1, 0].set_title('Spread Percentage by Exchange')
    axes[1, 0].set_ylabel('Spread (%)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Bid-Ask価格帯の比較
    x_pos = np.arange(len(current_prices))
    axes[1, 1].bar(x_pos, current_prices['bid'], label='Bid', alpha=0.7, color='blue')
    axes[1, 1].bar(x_pos, current_prices['ask'], label='Ask', alpha=0.7, color='red')
    axes[1, 1].set_title('Bid-Ask Comparison')
    axes[1, 1].set_ylabel('Price (JPY)')
    axes[1, 1].set_xticks(x_pos)
    axes[1, 1].set_xticklabels(current_prices['exchange'], rotation=45)
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # 価格差の統計
    print("\n価格差統計:")
    print(f"Ask最高: {current_prices.loc[current_prices['ask'].idxmax(), 'exchange']} - ¥{current_prices['ask'].max():,.0f}")
    print(f"Ask最低: {current_prices.loc[current_prices['ask'].idxmin(), 'exchange']} - ¥{current_prices['ask'].min():,.0f}")
    print(f"Ask価格差: ¥{current_prices['ask'].max() - current_prices['ask'].min():,.0f}")
    print(f"Bid最高: {current_prices.loc[current_prices['bid'].idxmax(), 'exchange']} - ¥{current_prices['bid'].max():,.0f}")
    print(f"Bid最低: {current_prices.loc[current_prices['bid'].idxmin(), 'exchange']} - ¥{current_prices['bid'].min():,.0f}")
    print(f"Bid価格差: ¥{current_prices['bid'].max() - current_prices['bid'].min():,.0f}")
else:
    print("現在の価格データがありません")

## 4. アービトラージ機会の計算

In [ ]:
def calculate_arbitrage_opportunities(prices_df):
    """アービトラージ機会を計算"""
    if len(prices_df) < 2:
        return pd.DataFrame()
    
    opportunities = []
    
    for i, buy_row in prices_df.iterrows():
        for j, sell_row in prices_df.iterrows():
            if i != j:  # 同じ取引所は除外
                # 買い（ask）と売り（bid）の差を計算
                profit = sell_row['bid'] - buy_row['ask']
                profit_pct = (profit / buy_row['ask']) * 100
                
                if profit > 0:
                    opportunities.append({
                        'buy_exchange': buy_row['exchange'],
                        'sell_exchange': sell_row['exchange'],
                        'buy_price': buy_row['ask'],
                        'sell_price': sell_row['bid'],
                        'profit': profit,
                        'profit_pct': profit_pct
                    })
    
    return pd.DataFrame(opportunities).sort_values('profit_pct', ascending=False)

# アービトラージ機会を計算
arbitrage_opps = calculate_arbitrage_opportunities(current_prices)

if not arbitrage_opps.empty:
    print("現在のアービトラージ機会:")
    display(arbitrage_opps.round(4))
    
    # アービトラージ機会の可視化
    if len(arbitrage_opps) > 0:
        plt.figure(figsize=(12, 6))
        
        # 利益率の棒グラフ
        plt.subplot(1, 2, 1)
        labels = [f"{row['buy_exchange']}→{row['sell_exchange']}" for _, row in arbitrage_opps.iterrows()]
        plt.bar(range(len(arbitrage_opps)), arbitrage_opps['profit_pct'], color='green', alpha=0.7)
        plt.title('Arbitrage Opportunities (Profit %)')
        plt.xlabel('Exchange Pair')
        plt.ylabel('Profit (%)')
        plt.xticks(range(len(arbitrage_opps)), labels, rotation=45)
        
        # 利益額の棒グラフ
        plt.subplot(1, 2, 2)
        plt.bar(range(len(arbitrage_opps)), arbitrage_opps['profit'], color='blue', alpha=0.7)
        plt.title('Arbitrage Opportunities (Profit Amount)')
        plt.xlabel('Exchange Pair')
        plt.ylabel('Profit (JPY)')
        plt.xticks(range(len(arbitrage_opps)), labels, rotation=45)
        
        plt.tight_layout()
        plt.show()
else:
    print("現在アービトラージ機会はありません")

## 5. 過去の価格推移分析

In [ ]:
def get_historical_prices(pair_id, hours=24, exchange_ids=None):
    """過去の価格データを取得"""
    with db.get_session() as session:
        start_time = datetime.now(jst) - timedelta(hours=hours)
        
        query = session.query(PriceTick).filter(
            PriceTick.pair_id == pair_id,
            PriceTick.timestamp > start_time
        )
        
        if exchange_ids:
            query = query.filter(PriceTick.exchange_id.in_(exchange_ids))
        
        ticks = query.order_by(PriceTick.timestamp).all()
        
        data = []
        for tick in ticks:
            data.append({
                'timestamp': tick.timestamp,
                'exchange_id': tick.exchange_id,
                'exchange': exchange_dict[tick.exchange_id],
                'bid': float(tick.bid),
                'ask': float(tick.ask),
                'mid': (float(tick.bid) + float(tick.ask)) / 2
            })
        
        return pd.DataFrame(data)

# 過去24時間のデータを取得
active_exchange_ids = [ex.id for ex in exchanges if ex.code != 'binance']
historical_prices = get_historical_prices(pair.id, hours=24, exchange_ids=active_exchange_ids)

if not historical_prices.empty:
    print(f"取得したデータ数: {len(historical_prices)}")
    print(f"時間範囲: {historical_prices['timestamp'].min()} - {historical_prices['timestamp'].max()}")
    
    # 価格推移の可視化
    plt.figure(figsize=(15, 10))
    
    # 1. Mid価格の推移
    plt.subplot(2, 2, 1)
    for exchange_name in historical_prices['exchange'].unique():
        exchange_data = historical_prices[historical_prices['exchange'] == exchange_name]
        plt.plot(exchange_data['timestamp'], exchange_data['mid'], label=exchange_name, marker='o', markersize=2)
    
    plt.title('Mid Price Trend (24h)')
    plt.xlabel('Time')
    plt.ylabel('Price (JPY)')
    plt.legend()
    plt.xticks(rotation=45)
    
    # 2. Bid価格の推移
    plt.subplot(2, 2, 2)
    for exchange_name in historical_prices['exchange'].unique():
        exchange_data = historical_prices[historical_prices['exchange'] == exchange_name]
        plt.plot(exchange_data['timestamp'], exchange_data['bid'], label=exchange_name, marker='o', markersize=2)
    
    plt.title('Bid Price Trend (24h)')
    plt.xlabel('Time')
    plt.ylabel('Price (JPY)')
    plt.legend()
    plt.xticks(rotation=45)
    
    # 3. Ask価格の推移
    plt.subplot(2, 2, 3)
    for exchange_name in historical_prices['exchange'].unique():
        exchange_data = historical_prices[historical_prices['exchange'] == exchange_name]
        plt.plot(exchange_data['timestamp'], exchange_data['ask'], label=exchange_name, marker='o', markersize=2)
    
    plt.title('Ask Price Trend (24h)')
    plt.xlabel('Time')
    plt.ylabel('Price (JPY)')
    plt.legend()
    plt.xticks(rotation=45)
    
    # 4. 価格差の推移
    plt.subplot(2, 2, 4)
    # 時間別の価格差を計算
    pivot_mid = historical_prices.pivot_table(index='timestamp', columns='exchange', values='mid')
    if len(pivot_mid.columns) >= 2:
        price_diff = pivot_mid.max(axis=1) - pivot_mid.min(axis=1)
        plt.plot(price_diff.index, price_diff.values, color='red', linewidth=2)
        plt.title('Price Difference (Max - Min)')
        plt.xlabel('Time')
        plt.ylabel('Price Difference (JPY)')
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print("\n価格統計 (24時間):")
    price_stats = historical_prices.groupby('exchange').agg({
        'mid': ['mean', 'std', 'min', 'max'],
        'bid': ['mean', 'min', 'max'],
        'ask': ['mean', 'min', 'max']
    }).round(2)
    
    display(price_stats)
else:
    print("過去のデータがありません")

## 6. アービトラージ機会の分析

In [ ]:
def get_arbitrage_history(pair_id, hours=24):
    """過去のアービトラージ機会を取得"""
    with db.get_session() as session:
        start_time = datetime.now(jst) - timedelta(hours=hours)
        
        opportunities = session.query(ArbitrageOpportunity).filter(
            ArbitrageOpportunity.pair_id == pair_id,
            ArbitrageOpportunity.timestamp > start_time
        ).order_by(ArbitrageOpportunity.timestamp).all()
        
        data = []
        for opp in opportunities:
            buy_ex = session.query(Exchange).filter_by(id=opp.buy_exchange_id).first()
            sell_ex = session.query(Exchange).filter_by(id=opp.sell_exchange_id).first()
            
            if buy_ex and sell_ex:
                data.append({
                    'timestamp': opp.timestamp,
                    'buy_exchange': buy_ex.name,
                    'sell_exchange': sell_ex.name,
                    'buy_price': float(opp.buy_price),
                    'sell_price': float(opp.sell_price),
                    'profit_pct': float(opp.estimated_profit_pct),
                    'status': opp.status
                })
        
        return pd.DataFrame(data)

# 過去24時間のアービトラージ機会を取得
arbitrage_history = get_arbitrage_history(pair.id, hours=24)

if not arbitrage_history.empty:
    print(f"検出されたアービトラージ機会数: {len(arbitrage_history)}")
    
    # アービトラージ機会の可視化
    plt.figure(figsize=(15, 12))
    
    # 1. 時間別のアービトラージ機会数
    plt.subplot(3, 2, 1)
    hourly_count = arbitrage_history.set_index('timestamp').resample('H').size()
    plt.bar(range(len(hourly_count)), hourly_count.values, color='skyblue')
    plt.title('Hourly Arbitrage Opportunities')
    plt.xlabel('Hour')
    plt.ylabel('Count')
    plt.xticks(range(len(hourly_count)), [t.strftime('%H:%M') for t in hourly_count.index], rotation=45)
    
    # 2. 利益率の分布
    plt.subplot(3, 2, 2)
    plt.hist(arbitrage_history['profit_pct'], bins=30, color='lightgreen', alpha=0.7)
    plt.title('Profit Percentage Distribution')
    plt.xlabel('Profit (%)')
    plt.ylabel('Frequency')
    
    # 3. 取引所ペア別の機会数
    plt.subplot(3, 2, 3)
    pair_counts = arbitrage_history.groupby(['buy_exchange', 'sell_exchange']).size().reset_index(name='count')
    pair_counts['pair'] = pair_counts['buy_exchange'] + ' → ' + pair_counts['sell_exchange']
    top_pairs = pair_counts.nlargest(10, 'count')
    
    plt.bar(range(len(top_pairs)), top_pairs['count'], color='orange')
    plt.title('Top 10 Exchange Pairs')
    plt.xlabel('Exchange Pair')
    plt.ylabel('Opportunities')
    plt.xticks(range(len(top_pairs)), top_pairs['pair'], rotation=45)
    
    # 4. 時間別の利益率推移
    plt.subplot(3, 2, 4)
    plt.scatter(arbitrage_history['timestamp'], arbitrage_history['profit_pct'], alpha=0.6, color='red')
    plt.title('Profit Percentage Over Time')
    plt.xlabel('Time')
    plt.ylabel('Profit (%)')
    plt.xticks(rotation=45)
    
    # 5. 取引所別の買いポジション数
    plt.subplot(3, 2, 5)
    buy_counts = arbitrage_history['buy_exchange'].value_counts()
    plt.pie(buy_counts.values, labels=buy_counts.index, autopct='%1.1f%%')
    plt.title('Buy Exchange Distribution')
    
    # 6. 取引所別の売りポジション数
    plt.subplot(3, 2, 6)
    sell_counts = arbitrage_history['sell_exchange'].value_counts()
    plt.pie(sell_counts.values, labels=sell_counts.index, autopct='%1.1f%%')
    plt.title('Sell Exchange Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print("\nアービトラージ統計 (24時間):")
    print(f"合計機会数: {len(arbitrage_history)}")
    print(f"平均利益率: {arbitrage_history['profit_pct'].mean():.4f}%")
    print(f"最大利益率: {arbitrage_history['profit_pct'].max():.4f}%")
    print(f"標準偏差: {arbitrage_history['profit_pct'].std():.4f}%")
    
    # 時間別統計
    hourly_stats = arbitrage_history.set_index('timestamp').resample('H').agg({
        'profit_pct': ['count', 'mean', 'max']
    }).round(4)
    
    print("\n時間別統計:")
    display(hourly_stats)
    
else:
    print("過去24時間にアービトラージ機会は検出されませんでした")

## 7. リアルタイム監視機能

In [ ]:
# リアルタイム監視のためのヘルパー関数
def monitor_current_situation():
    """現在の状況を監視"""
    print("="*60)
    print(f"現在時刻: {datetime.now(jst).strftime('%Y-%m-%d %H:%M:%S')} JST")
    print("="*60)
    
    # 現在の価格を取得
    current_prices = get_current_prices(pair.id)
    
    if not current_prices.empty:
        print("\n現在の価格:")
        for _, row in current_prices.iterrows():
            print(f"{row['exchange']:^12}: Bid ¥{row['bid']:,.0f} | Ask ¥{row['ask']:,.0f}")
        
        # アービトラージ機会を計算
        arbitrage_opps = calculate_arbitrage_opportunities(current_prices)
        
        if not arbitrage_opps.empty:
            print("\n🚀 アービトラージ機会:")
            for _, row in arbitrage_opps.iterrows():
                print(f"{row['buy_exchange']} → {row['sell_exchange']}: {row['profit_pct']:.3f}% (¥{row['profit']:,.0f})")
        else:
            print("\n現在アービトラージ機会はありません")
    else:
        print("\n価格データがありません")
    
    # 過去15分間の機会数
    recent_opps = get_arbitrage_history(pair.id, hours=0.25)  # 15分
    if not recent_opps.empty:
        print(f"\n過去15分間の機会数: {len(recent_opps)}")
        if len(recent_opps) > 0:
            print(f"最大利益率: {recent_opps['profit_pct'].max():.3f}%")
    else:
        print("\n過去15分間にアービトラージ機会はありませんでした")

# 現在の状況を表示
monitor_current_situation()

## 8. データ品質チェック

In [ ]:
def data_quality_check():
    """データ品質をチェック"""
    print("データ品質チェック")
    print("="*50)
    
    with db.get_session() as session:
        # 1. 各取引所の最新データ取得時刻
        print("\n1. 最新データ取得時刻:")
        for exchange in exchanges:
            if exchange.code == 'binance':
                continue
                
            latest_tick = session.query(PriceTick).filter_by(
                exchange_id=exchange.id,
                pair_id=pair.id
            ).order_by(PriceTick.timestamp.desc()).first()
            
            if latest_tick:
                time_diff = datetime.now(jst) - latest_tick.timestamp
                status = "🟢" if time_diff < timedelta(minutes=5) else "🔴"
                print(f"{status} {exchange.name:^12}: {latest_tick.timestamp.strftime('%H:%M:%S')} ({time_diff})")
            else:
                print(f"🔴 {exchange.name:^12}: データなし")
        
        # 2. 過去1時間のデータ取得状況
        print("\n2. 過去1時間のデータ取得数:")
        one_hour_ago = datetime.now(jst) - timedelta(hours=1)
        
        for exchange in exchanges:
            if exchange.code == 'binance':
                continue
                
            tick_count = session.query(PriceTick).filter(
                PriceTick.exchange_id == exchange.id,
                PriceTick.pair_id == pair.id,
                PriceTick.timestamp > one_hour_ago
            ).count()
            
            expected_count = 3600 / 5  # 5秒間隔での期待値
            coverage = (tick_count / expected_count) * 100
            status = "🟢" if coverage > 80 else "🟡" if coverage > 50 else "🔴"
            print(f"{status} {exchange.name:^12}: {tick_count:^5}件 ({coverage:.1f}%)")
        
        # 3. アービトラージ検出状況
        print("\n3. アービトラージ検出状況:")
        arb_count = session.query(ArbitrageOpportunity).filter(
            ArbitrageOpportunity.pair_id == pair.id,
            ArbitrageOpportunity.timestamp > one_hour_ago
        ).count()
        
        print(f"過去1時間の検出数: {arb_count}件")
        
        if arb_count > 0:
            avg_profit = session.query(func.avg(ArbitrageOpportunity.estimated_profit_pct)).filter(
                ArbitrageOpportunity.pair_id == pair.id,
                ArbitrageOpportunity.timestamp > one_hour_ago
            ).scalar()
            
            max_profit = session.query(func.max(ArbitrageOpportunity.estimated_profit_pct)).filter(
                ArbitrageOpportunity.pair_id == pair.id,
                ArbitrageOpportunity.timestamp > one_hour_ago
            ).scalar()
            
            print(f"平均利益率: {float(avg_profit):.3f}%")
            print(f"最大利益率: {float(max_profit):.3f}%")

# データ品質チェック実行
data_quality_check()

## 9. 実行可能なスクリプト

このノートブックと同じ機能をターミナルから実行できるスクリプトも用意されています：

```bash
# 現在の状況を確認
python scripts/check_arbitrage.py

# リアルタイム監視
python scripts/monitor_arbitrage.py

# オプション付きで実行
python scripts/monitor_arbitrage.py --interval 3 --threshold 0.05
```